## 기본 구조: 프롬프트 + 모델 + 출력 파서

가장 기본적인 사용 구조는
prompt 템플릿과 모델을 함께 연결하는 것입니다.

이것이 어떻게 동작하는지 보기 위해, 각 나라별 수도를 물어보는 Chain을 생성해 보겠습니다.

In [ ]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

## 프롬프트 템플릿의 활용

**PromptTemplate**
+ 사용자의 입력 변수를 주입하여 완전한 프롬프트 문자열을 만드는 데 사용되는 템플릿.
+ 사용법:
    - **template**: 템플릿 문자열. 이 문자열에는 중괄호 {}가 변수를 나타냄.
    - **input_variable**: 중괄호 안에 들어갈 변수의 이름을 리스트로 정의.


### input_variable
- input_varaible는 PromptTemplate에서 사용되는 변수의 이름을 정의하는 리스트.

In [ ]:
from langchain_core.prompts import PromptTemplate

.from_template() 메소드를 사용하여 PromptTemplate 객체 생성

In [ ]:
# template 정의
template = "{country}의 수도는 어디인가요?"

# from_template 메소드를 이용하여 PromptTemplate 객체 생성
prompt_template = PromptTemplate.from_template(template)
prompt_template

이제 PromptTemplate 객체를 사용하여 완전한, Prompt를 생성해보자.

만약, PromptTemplate의 input_variables에 값을 주입하지 않는다면?

In [ ]:
# prompt 생성
prompt = prompt_template.format(country='')
prompt

PromptTemplate의 input_variables에 값을 주입한 경우

In [ ]:
# prompt 생성
prompt = prompt_template.format(country='대한민국')
prompt

※ 실습: input_variables로 영어 문장으로 받고, 한국어 번역을 요청하는 prompt를 생성하시오.
- PromptTemplate 라이브러리 임포트
- template 정의
    * input_variables: 'en_dialogue'
- prompt_template의 input_variables에 다음 영어 문장을 주입하여 prompt 생성
    * 'It is only with the heart that one can see rightly; what is essential is invisible to the eye. - My Litte Prince'
- prompt 출력

In [ ]:
# YOUR CODE

## Chain 생성
LCEL (LangChain Expression Language)
![LCEL](https://codetutorbot.blob.core.windows.net/image/LCEL.png)

- LCEL을 사용, 구성 요소를 단일 체인으로 결합할 수 있음.

### chain's syntax:
- chain = prompt | llm | output_parser

'|'는 파이프 연산자, 서로 다른 구성 요소를 연결하고 한 구성 요소의 '출력'을 다음 구성 요소의 '입력'으로 전달.

### chain's workflow:
- input_variables => PromptTemplate 에 전달
- PromptTemplate => LLM 에 전달
- LLM's generated response => output_parser 에 전달

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 요약을 위한 프롬프트 템플릿 문자열 정의
template = "{text_contents}를 30글자 이내로 요약해줘"

# PromptTemplate 객체 생성
prompt_template = PromptTemplate.from_template(template)

# OpenAI의 챗 모델 인스턴스 생성 (기본값은 gpt-3.5-turbo)
llm = ChatOpenAI()

# 프롬프트 템플릿과 LLM을 파이프 연산자로 연결하여 체인 구성
chain = prompt_template | llm


### invoke() 호출
- python 딕셔너리 형태로 입력값을 전달합니다.(키: 값)
- invoke() 함수 호출 시, 입력값을 전달합니다.

In [ ]:
# input 딕셔너리에 요약할 문장 저장
input = {
    'text_contents': '''
        A large language model (LLM) is a type of machine learning model designed for natural language processing tasks such as language generation. LLMs are language models with many parameters, and are trained with self-supervised learning on a vast amount of text.

        The largest and most capable LLMs are generative pretrained transformers (GPTs). Modern models can be fine-tuned for specific tasks or guided by prompt engineering.[1] These models acquire predictive power regarding syntax, semantics, and ontologies[2] inherent in human language corpora, but they also inherit inaccuracies and biases present in the data they are trained in.
        '''
}

In [ ]:
# prompt 객체와 model 객체를 파이프(|) 연산자로 연결하고 invoke 메서드를 사용하여 input을 전달
# 이를 통해 AI 모델이 생성한 메시지를 반환
chain.invoke(input)

※ 실습: input_variables로 topic을 받고, 100글자 이내로 설명하는 chain을 생성하시오.
- PromptTemplate, ChatOpenAI 라이브러리 임포트
- template 정의
    * topic을 설명하기 위한 프롬프트 템플릿 문자열 정의
    * input_variables: 'topic'
- 챗 모델 인스턴스 생성
    * max_tokens: 100
    * model_name: gpt-4o-mini
- propmpt_template | llm의 chain 생성
- chain invoke
    * input = {
    'topic': '4차 산업 혁명'
}

In [ ]:
# YOUR CODE

## 출력파서 (Output Parser)
출력파서를 이용하면, AIMessage로 받아오는 결과에서 content 부분만 잘라서 가져올 수 있다.

편리하기 때문에 사용.


In [ ]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
# 출력파서 라이브러리 추가
from langchain_core.output_parsers import StrOutputParser

# output_parser 객체 생성
output_parser = StrOutputParser()


template = "{text_contents}를 30글자 이내로 요약해줘"
prompt_template = PromptTemplate.from_template(template)

llm = ChatOpenAI(model_name='gpt-4o-mini')

input = {
    'text_contents': '''
        A large language model (LLM) is a type of machine learning model designed for natural language processing tasks such as language generation. LLMs are language models with many parameters, and are trained with self-supervised learning on a vast amount of text.

        The largest and most capable LLMs are generative pretrained transformers (GPTs). Modern models can be fine-tuned for specific tasks or guided by prompt engineering.[1] These models acquire predictive power regarding syntax, semantics, and ontologies[2] inherent in human language corpora, but they also inherit inaccuracies and biases present in the data they are trained in.
        '''
}

# chain에 output_parser 추가
chain = prompt_template | llm | output_parser

chain.invoke(input)


※ 실습: 다음은 영어회화 포맷을 요청하는 재밌는 예시이다. 해당 코드를 직접 받아 작성하며, chain.invoke() 과정과 다양한 템플릿을 익혀보자.
![LCEL_exp](https://codetutorbot.blob.core.windows.net/image/lcelexample.png)

In [ ]:
#YOUR CODE